# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [2]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:
# percentage of data used for testing
TEST_PERCENTAGE = 0.20
# model learning rate
LEARNING_RATE = 0.1
# no of epochs to fit model
NO_EPOCHS = 80

NUM_COMPONENTS = 30

NUM_THREADS = 4

ALPHA = 1e-3

MAX_SAMPLED = 10

# seed for pseudonumber generations
SEED = 42

### Retrieve data

In [4]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [5]:
import pandas as pd

df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=500000)
df_ratings

,Unnamed: 0,userId,history,userType,score
0,0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...,...
6349891,6349891,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,7a349b09-badc-40a9-a194-83d959aeb50c,Non-Logged,1.94055
6349892,6349892,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,6f344c45-e731-41b4-8c65-9967ebc03096,Non-Logged,3.361101
6349893,6349893,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,4c586bb4-f71d-4b39-9df8-e38ac3f632a0,Non-Logged,0.919598
6349894,6349894,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,855d20b7-53f2-4678-a10f-55402d085018,Non-Logged,1.990197


In [6]:
df_ratings.drop(columns=["Unnamed: 0"],inplace=True)
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6349896 entries, 0 to 6349895
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userId    string  
 1   history   string  
 2   userType  category
 3   score     Float32 
dtypes: Float32(1), category(1), string(2)
memory usage: 133.2 MB


In [7]:
df_ratings.describe()

,score
count,6349896.0
mean,1.608734
std,0.867295
min,0.002373
25%,0.924991
50%,1.56746
75%,2.202917
max,4.990057


In [8]:
dtype_df_items = {
"page" : 'string',
"url" : 'string',
"issued" : 'string',
"modified" : 'string',
"title" : 'string',
"body" : 'string',
"caption" : 'string',
"age_in_days" : 'UInt32',
"age_exp" : 'Float32',
"age_exp_normalized" : 'Float32',
"ageCategories" : 'category'
}

In [9]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"], dtype=dtype_df_items)
df_news.drop(columns=["Unnamed: 0"],inplace=True)

In [10]:
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.12261,very-old


In [11]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255603 entries, 0 to 255602
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   page                255603 non-null  string  
 1   age_exp_normalized  255603 non-null  Float32 
 2   ageCategories       255603 non-null  category
dtypes: Float32(1), category(1), string(1)
memory usage: 3.4 MB


In [12]:
df_merged = pd.merge(df_ratings, df_news, left_on='history', right_on='page', how='left')
df_merged.drop(columns=["page"],inplace=True) # 'page' is the same as 'history'

In [13]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6349896 entries, 0 to 6349895
Data columns (total 6 columns):
 #   Column              Dtype   
---  ------              -----   
 0   userId              string  
 1   history             string  
 2   userType            category
 3   score               Float32 
 4   age_exp_normalized  Float32 
 5   ageCategories       category
dtypes: Float32(2), category(2), string(2)
memory usage: 169.6 MB


In [14]:
df_merged.head()

,userId,history,userType,score,age_exp_normalized,ageCategories
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477,0.980416,recent
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501,0.613061,mid
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861,0.880859,recent
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271,0.945895,recent
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852,0.13293,very-old


In [15]:
w_initial_score = 0.85
w_age_norm = 1-w_initial_score
df_merged["score"] = df_merged["score"]*w_initial_score + df_merged["age_exp_normalized"]*w_age_norm
df_merged["score"] = df_merged["score"]/df_merged["score"].max()
df_merged.describe()

,score,age_exp_normalized
count,6349896.0,6349896.0
mean,0.342372,0.900751
std,0.168628,0.183626
min,0.01394,0.102536
25%,0.209248,0.934135
50%,0.334428,0.956106
75%,0.457999,0.978657
max,1.0,1.0


In [16]:
df_merged = df_merged[["userId","history","userType","score"]]

df_merged.rename(columns={"ageCategories" : "historyFreshnessNormalized"},inplace=True)
df_merged.describe()

,score
count,6349896.0
mean,0.342372
std,0.168628
min,0.01394
25%,0.209248
50%,0.334428
75%,0.457999
max,1.0


In [17]:
# df_merged.head()
df_merged.head()

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,0.462804
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,0.491507
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,0.377742
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,0.358243
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,0.443595


In [18]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6349896 entries, 0 to 6349895
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userId    string  
 1   history   string  
 2   userType  category
 3   score     Float32 
dtypes: Float32(1), category(1), string(2)
memory usage: 133.2 MB


### Prepare data

Before fitting the LightFM model, we need to create an instance of `Dataset` which holds the interaction matrix.

In [19]:
dataset = Dataset()

# Get unique values for users, items, and user features
unique_users = df_merged["userId"].unique()
unique_items = df_merged["history"].unique()
unique_user_features = df_merged["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset.fit(
    users=unique_users,
    items=unique_items,
    user_features=unique_user_features  # Register user features
)


In [20]:
(interactions, weights) = dataset.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_merged.iterrows()
])


In [21]:
user_features_list = [
    (row.userId, [row.userType])  
    for _, row in df_merged.iterrows()
]

user_features = dataset.build_user_features(user_features_list)


LightLM works slightly differently compared to other packages as it expects the train and test sets to have same dimension. Therefore the conventional train test split will not work.

The package has included the `cross_validation.random_train_test_split` method to split the interaction data and splits it into two disjoint training and test sets. 

However, note that **it does not validate the interactions in the test set to guarantee all items and users have historical interactions in the training set**. Therefore this may result into a partial cold-start problem in the test set.

In [22]:
# Split train and test sets (80/20 split)
train, test = cross_validation.random_train_test_split(interactions, test_percentage=TEST_PERCENTAGE, random_state=SEED)
train_weights, test_weights  = cross_validation.random_train_test_split(weights, test_percentage=TEST_PERCENTAGE, random_state=SEED)


Double check the size of both the train and test sets.

In [23]:
print(f"Shape of train interactions: {train.shape}")
print(f"Shape of test interactions: {test.shape}")

Shape of train interactions: (532037, 231305)
Shape of test interactions: (532037, 231305)


In [24]:
print(f"Shape of train interactions: {train_weights.shape}")
print(f"Shape of test interactions: {test_weights.shape}")

Shape of train interactions: (532037, 231305)
Shape of test interactions: (532037, 231305)


### Fit the LightFM model

In this notebook, the LightFM model will be using the weighted Approximate-Rank Pairwise (WARP) as the loss. Further explanation on the topic can be found [here](https://making.lyst.com/lightfm/docs/examples/warp_loss.html#learning-to-rank-using-the-warp-loss).


In general, it maximises the rank of positive examples by repeatedly sampling negative examples until a rank violation has been located. This approach is recommended when only positive interactions are present.

The LightFM model can be fitted with the following code:

In [ ]:
model = LightFM(no_components=NUM_COMPONENTS,loss="warp",learning_rate=LEARNING_RATE,user_alpha=ALPHA,max_sampled=MAX_SAMPLED,random_state=np.random.RandomState(SEED))  # Weighted Approximate-Rank Pairwise (WARP) loss
model.fit(train, sample_weight=train_weights, epochs=NO_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)
# model.fit(train, epochs=NO_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)
# model.fit(interactions, sample_weight=weights, epochs=NO_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)


### Evaluate model

In [26]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute evaluation metrics
auc_train = auc_score(model, train, user_features=user_features, num_threads=NUM_THREADS).mean()
auc_test = auc_score(model, test, train_interactions=train, user_features=user_features, num_threads=NUM_THREADS).mean()

# Print evaluation results
print(f"AUC test Score: {auc_test:.4f}")
print(f"AUC train Score: {auc_train:.4f}")

AUC test Score: 0.8595
AUC train Score: 0.8418


### Save pkls to serve model

In [27]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [28]:
item_id_map_reverse = {v: k for k, v in item_id_map.items()}

In [29]:
import pickle

pickle.dump(model, open('artifacts/lightfm_model.pkl', 'wb'))
pickle.dump(user_id_map, open('artifacts/user_id_map.pkl', 'wb'))
pickle.dump(item_id_map_reverse, open('artifacts/item_id_map_reverse.pkl', 'wb'))
pickle.dump(user_feature_map, open('artifacts/user_feature_map.pkl', 'wb'))

In [30]:
loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))

### Make predictions to known and unknowm on same recommendation function with pkls

In [31]:

_, n_items = interactions.shape # no of users * no of items

n_items

231305

In [32]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [33]:
def sample_recommendation(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:5]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    print("Top 5 recommended items:")

    for x in top_5_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["page"].values[0])


In [34]:
# # predict for known user
# user_feature_list = ['userType:Logged']
# user_hash = '5f5e17781fc2ec0ddcfb2e9356e61c5d3d4b0b3c8fabd20917feb9e807463856'
# sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

In [35]:
# predict for unknown user
user_feature_list = ['userType:Non-Logged']
user_hash = ''
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

new user feature encountered 'userType:Non-Logged'
Top 5 recommended items:
        d2593c3d-2347-40d9-948c-b6065e8459a9
        f6b5d170-48b9-4f8e-88d4-c84b6668f3bd
        1f32787b-de2b-49be-8c20-ddaeae34cc22
        6a83890a-d9e9-4f6b-a6c6-90d031785bbf
        4c63d7cd-4902-4ffb-9b94-578b1b2151f0
